In [147]:
allocator <- function(units, preds) {
    zones <- c('zone1', 'zone2', 'zone3', 'zone4', 'zone5', 'zone6', 'zone7')
    total <- sum(preds)
    if (units <= 7) {  # if fewer units than zones 
        preds <- sort(preds, decreasing = TRUE)
        alloc <- rep(0, 7)
        names(alloc) <- zones
        alloc[names(preds[1: units])] <- 1  # assign to zones with greatest demand
        return (alloc)
    }
    else {
        alloc <- rep(1, 7)  # give each zone a unit
        names(alloc) <- zones
        units <- units - 7
        additional <- floor((preds * units) / total)  # distribute units based upon need
        alloc <- alloc + additional
        units <- units - sum(additional)
        if (units > 0){  # if any units are unallocated
            sorted_alloc <- sort(alloc)  # assign remaining units to zones with fewest units
            alloc[names(sorted_alloc[1: units])] <- alloc[names(sorted_alloc[1: units])] + 1
        }
        return (alloc)
    }
}

In [151]:
units <- 36
zones <- c('zone1', 'zone2', 'zone3', 'zone4', 'zone5', 'zone6', 'zone7')
preds <- c(10, 40, 100, 2, 0, 77, 80)
names(preds) <- zones

In [15]:
library(geosphere)
library(fossil)
library(flexclust)

Loading required package: maps
Loading required package: shapefiles
Loading required package: foreign

Attaching package: ‘shapefiles’

The following objects are masked from ‘package:foreign’:

    read.dbf, write.dbf



In [9]:
distHaversine()

ERROR: Error in inherits(p, "SpatialPoints"): argument "p1" is missing, with no default


In [1]:
dta <- read.csv('../data/seattle_911_geo_time_features.csv')
head(dta)

Address,Type,Latitude,Longitude,Timestamp,Date,Time,Weekday,Monday,Tuesday,⋯,Night,DangHoliday,TravHoliday,DayNumber,Seasonality,MarinersHome,SeahawksHome,SoundersHome,Neighborhood,Zone
6900 37th Av S,Medic Response,47.54068,-122.2861,2011-11-09 23:33:00,2011-11-10,23:33:00,Thursday,0,0,⋯,1,0,0,0.1886578,0.4362022,0,0,0,Holly Park,7
N 50th St / Stone Way N,Aid Response,47.66503,-122.3402,2011-11-09 23:32:00,2011-11-10,23:32:00,Thursday,0,0,⋯,1,0,0,0.1886578,0.4362022,0,0,0,Wallingford,1
E John St / E Olive Way,Aid Response,47.61957,-122.3243,2011-11-09 23:32:00,2011-11-10,23:32:00,Thursday,0,0,⋯,1,0,0,0.1886578,0.4362022,0,0,0,Broadway,4
611 12th Av S,Aid Response,47.59741,-122.3172,2011-11-09 23:29:00,2011-11-10,23:29:00,Thursday,0,0,⋯,1,0,0,0.1886578,0.4362022,0,0,0,International District,5
4545 42nd Av Sw,Automatic Medical Alarm,47.56247,-122.3855,2011-11-09 23:25:00,2011-11-10,23:25:00,Thursday,0,0,⋯,1,0,0,0.1886578,0.4362022,0,0,0,Genesee,6
2124 3rd Av,Investigate Out Of Service,47.61335,-122.3425,2011-11-09 23:17:00,2011-11-10,23:17:00,Thursday,0,0,⋯,1,0,0,0.1886578,0.4362022,0,0,0,Belltown,5


In [49]:
x <- subset(dta, select = c(4,3))[1:500,]

In [52]:
centroid_finder <- function(x, k) {
    d <- earth.dist(x)
    km <- kmeans(d, centers = k)
    x$cluster <- km$cluster
    return (aggregate(.~cluster, data = x, mean)[, c(1,3,2)])
} 

In [53]:
centroid_finder(x, 5)


cluster,Latitude,Longitude
1,47.65436,-122.3326
2,47.71866,-122.3275
3,47.61026,-122.3292
4,47.68106,-122.3433
5,47.54770,-122.3270


In [38]:
x$label <- km$cluster
head(x)

Longitude,Latitude,label
-122.2861,47.54068,3
-122.3402,47.66503,2
-122.3243,47.61957,1
-122.3172,47.59741,1
-122.3855,47.56247,3
-122.3425,47.61335,1


In [42]:
aggregate(.~label, data = x, mean)


label,Longitude,Latitude
1,-122.3301,47.61245
2,-122.3351,47.68453
3,-122.3268,47.54794
